# Workshop. Neural networks' tools (Tensorflow)

<p style='text-align: right;font-style: italic; color: red;'>Designed by: Mr. Abdelkrime Aries</p>

In [1]:
# Sometimes, TensorFlow throws errors when there is no GPU. 
# To stop these messages, we can use this code:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.17.0'

In [3]:
import pandas     as pd
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

pd.__version__

'2.2.2'

In [4]:
from typing import Literal, List

## I. Data Preparation

In [5]:
train = pd.read_csv('data/sat.trn', delimiter=' ', header=None)

X_train = train.iloc[:, :-1].values
Y_train = train.iloc[:,  -1].values

lbin = LabelBinarizer()

X_train = X_train / 255.
Y_train = lbin.fit_transform(Y_train)

X_train = tf.constant(X_train, dtype=tf.float32)
Y_train = tf.constant(Y_train, dtype=tf.float32)

X_train.shape, Y_train.shape

(TensorShape([4435, 36]), TensorShape([4435, 6]))

In [6]:
test = pd.read_csv('data/sat.tst', delimiter=' ', header=None)

X_test = test.iloc[:, :-1].values
Y_test = test.iloc[:,  -1].values

X_test = X_test / 255.
# Y_test = lbin.transform(Y_test)

X_test = tf.constant(X_test, dtype=tf.float32)

X_test.shape, Y_test.shape

(TensorShape([2000, 36]), (2000,))

## II. Keras

### II.1. Sequential model

In [7]:
nn1 = keras.Sequential()
nn1.add(keras.Input(shape=(X_train.shape[1],)))
nn1.add(keras.layers.Dense(10, activation='relu'))
nn1.add(keras.layers.Dense(10, activation='relu'))
nn1.add(keras.layers.Dense(Y_train.shape[1], activation='softmax'))


nn1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 10)             │           370 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 546 (2.13 KB)

 Trainable params: 546 (2.13 KB)

 Non-trainable params: 0 (0.00 B)

### II.2. Model training

In [8]:
nn1.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01), 
    loss=keras.losses.CategoricalCrossentropy()
    )
nn1.fit(X_train, Y_train, epochs=100)

Epoch 1/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 509us/step - loss: 1.5745
Epoch 2/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step - loss: 0.8745
Epoch 3/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 389us/step - loss: 0.6218
Epoch 4/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 365us/step - loss: 0.5431
Epoch 5/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 371us/step - loss: 0.5262
Epoch 6/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 437us/step - loss: 0.5003
Epoch 7/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 377us/step - loss: 0.4901
Epoch 8/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 457us/step - loss: 0.4949
Epoch 9/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 476us/step - loss: 0.4927
Epoch 10/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - loss: 0.4731
Epoch 11/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step - loss: 0.4929
Epoch 12/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - loss: 0.4413
Epoch 13/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - loss: 0.4551
Epoch 14/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - loss: 0.4440
Epoch 15/100
13

### II.3. Model testing

In [9]:
print(classification_report(Y_test, lbin.inverse_transform(nn1(X_test).numpy()), zero_division=0))

              precision    recall  f1-score   support

           1       0.94      0.99      0.97       461
           2       0.93      0.95      0.94       224
           3       0.92      0.88      0.90       397
           4       0.53      0.61      0.57       211
           5       0.85      0.76      0.80       237
           7       0.84      0.81      0.82       470

    accuracy                           0.85      2000
   macro avg       0.83      0.83      0.83      2000
weighted avg       0.86      0.85      0.85      2000



## III. High level with a custom class

### III.1. Custom Layer

In [10]:
# MyLayer in here
class MyLayer(keras.layers.Dense):
    def __init__(self, 
                 nb_in:int, nb_out: int, 
                 bias: bool = True, act: Literal['relu', 'sigmoid', 'linear'] = 'linear'):
        assert nb_in   > 0
        assert nb_out  > 0
        super().__init__(nb_out, use_bias=bias, activation=act)

        self.build((nb_in,))


MyLayer(3, 2)

<MyLayer name=my_layer, built=True>

In [11]:
# Must print an 'Exception' or 'AssertionError'

try:
    ml1 = MyLayer(0, 2)
except Exception as e:
    print(repr(e))

print('end')

AssertionError()
end


In [12]:
l2ts = [
    MyLayer(3, 2, bias=False, act='relu'),
    MyLayer(3, 2, bias=True, act='sigmoid'),
    MyLayer(3, 1)
    ]

XX = tf.constant([[1, 2, 3], [4, 5, 6]])

for l in l2ts:
    print('===============================')
    print(l)
    print('-------------------------------')
    print('bias=', l.bias)
    weight = l.kernel
    print('output=', l(XX))

<MyLayer name=my_layer_1, built=True>
-------------------------------
bias= None
output= tf.Tensor(
[[0.43310857 1.5844113 ]
 [0.         3.8915799 ]], shape=(2, 2), dtype=float32)
<MyLayer name=my_layer_2, built=True>
-------------------------------
bias= <KerasVariable shape=(2,), dtype=float32, path=my_layer_2/bias>
output= tf.Tensor(
[[0.915719   0.19354774]
 [0.9928367  0.02089548]], shape=(2, 2), dtype=float32)
<MyLayer name=my_layer_3, built=True>
-------------------------------
bias= <KerasVariable shape=(1,), dtype=float32, path=my_layer_3/bias>
output= tf.Tensor(
[[-2.0745015]
 [-4.9210815]], shape=(2, 1), dtype=float32)


### III.2. Custom Net

In [13]:
class MyMLP(keras.Model):
    def __init__(self):
        super().__init__()
        self.layers_list = []
        self.locked = False
    
    def add_layer(self, layer: MyLayer):
        if self.locked:
            raise Exception('You cannot add more layers')
        out_nbr = None
        if len(self.layers_list):
            out_nbr = self.layers_list[-1].kernel.shape[1]
        in_nbr = layer.kernel.shape[0]
        if out_nbr is not None and out_nbr != in_nbr:
            raise Exception(f'The last layer outputs ({out_nbr}) must be the same as this layer input {in_nbr}')
        self.layers_list.append(layer)
        return self
        
    def compile(self, nb_in=1, nb_out=1, bias=True, multiclass=False, lr=1.):
        if len(self.layers_list):
            nb_in = self.layers_list[-1].kernel.shape[1]

        loss = keras.losses.BinaryCrossentropy()
        act = 'sigmoid'
        if multiclass and nb_out > 1:
            act='softmax'
            loss = keras.losses.CategoricalCrossentropy()
        self.layers_list.append(MyLayer(nb_in, nb_out, bias=bias, act=act))
        optimizer = keras.optimizers.Adam(learning_rate=0.01)
        self.locked = True
        super().compile(optimizer=optimizer, loss=loss)

    def forward(self, X):
        Z = X 
        for layer in self.layers_list:
            Z = layer(Z)
        return Z

    def __call__(self, X):
        return self.forward(X)

### III.3. Model training

In [14]:
nn2 = MyMLP()
nn2.add_layer(MyLayer(X_train.shape[1], 10, act='relu'))\
   .add_layer(MyLayer(10, 10, act='relu'))\
   .compile(nb_out=Y_train.shape[1], lr=0.01, multiclass=True)

nn2.summary()

Model: "my_mlp"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ my_layer_4 (MyLayer)            │ (10)                   │           370 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ my_layer_5 (MyLayer)            │ (10)                   │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ my_layer_6 (MyLayer)            │ (6)                    │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 546 (2.13 KB)

 Trainable params: 546 (2.13 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
nn2.fit(X_train, Y_train, epochs=100)

Epoch 1/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step - loss: 1.5031
Epoch 2/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step - loss: 0.6514
Epoch 3/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 484us/step - loss: 0.5156
Epoch 4/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 458us/step - loss: 0.5029
Epoch 5/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - loss: 0.4441
Epoch 6/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - loss: 0.4736
Epoch 7/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - loss: 0.4565
Epoch 8/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 395us/step - loss: 0.4526
Epoch 9/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - loss: 0.4673
Epoch 10/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 433us/step - loss: 0.4400
Epoch 11/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - loss: 0.4410
Epoch 12/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - loss: 0.4106
Epoch 13/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 402us/step - loss: 0.4218
Epoch 14/100
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - loss: 0.4121
Epoch 15/100
13

### III.4. Model testing

In [16]:
print(classification_report(Y_test, lbin.inverse_transform(nn2(X_test).numpy()), zero_division=0))

              precision    recall  f1-score   support

           1       0.98      0.97      0.98       461
           2       0.90      0.98      0.94       224
           3       0.71      0.98      0.83       397
           4       0.74      0.19      0.30       211
           5       0.93      0.66      0.77       237
           7       0.77      0.86      0.81       470

    accuracy                           0.83      2000
   macro avg       0.84      0.78      0.77      2000
weighted avg       0.84      0.83      0.81      2000



## IV. Low level

### IV.1. Activation functions

In [51]:
def simple_sigmoid(X):
    return 1/(1+tf.math.exp(-X))

def simple_ReLU(X):
    return tf.where(X > 0., X, 0.)
    
def simple_softmax(X):
    H = tf.math.exp(X)
    # return H/tf.math.reduce_sum(H, axis=0)
    return H/tf.reshape(tf.math.reduce_sum(H, axis=1), (-1, 1))


In [43]:
XX = tf.constant([[1., -1., 0.], [-0.5, 0.2, 5]])
print(simple_sigmoid(XX))
print(simple_ReLU(XX))
print(simple_softmax(XX))

tf.Tensor(
[[0.7310586  0.26894143 0.5       ]
 [0.37754068 0.54983395 0.9933072 ]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[1.  0.  0. ]
 [0.  0.2 5. ]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[0.81757444 0.2314752  0.00669285]
 [0.18242553 0.76852477 0.9933072 ]], shape=(2, 3), dtype=float32)


### IV.2. Loss functions

In [30]:
class SimpleBCE(keras.Loss):
    def call(self, H, Y):
        return tf.reduce_mean(- Y * tf.math.log(H) - (1-Y) * tf.math.log(1-H))
    
class SimpleCE(keras.Loss):
    def call(self, H, Y):
        return tf.reduce_mean(- Y * tf.math.log(H))

### IV.3. Optimization functions

In [31]:
class SimpleGD(keras.Optimizer):
    def __init__(self, learning_rate=0.001):
        super().__init__(learning_rate=learning_rate)
    def apply_gradients(self, grads_and_vars):
        for grads, vars in grads_and_vars:
            vars.assign_sub(self.learning_rate * grads)

### IV.4. Custom Layer

In [33]:
# SimpleLayer in here
class SimpleLayer(object):
    def __init__(self, 
                 nb_in: int, nb_out: int, 
                 bias: bool = True, act: Literal['relu', 'sigmoid', 'linear'] = 'linear'):
        assert nb_in   > 0
        assert nb_out  > 0
        super().__init__()

        self.W = tf.Variable(tf.zeros([nb_in, nb_out]))
        self.trainable_weights = [self.W]
        self.b = tf.zeros([1   , nb_out])
        if bias:
            self.b = tf.Variable(self.b)
            self.trainable_weights.append(self.b)

        self.act = lambda x: x
        if act == 'relu':
            self.act = simple_ReLU
        elif act == 'sigmoid':
            self.act = simple_sigmoid

    def randomize(self):
        self.W.assign(tf.random.normal(self.W.shape, mean=0.0, stddev=0.1))
        if isinstance(self.b, tf.Variable):
            self.b.assign(tf.random.normal(self.b.shape, mean=0.0, stddev=0.1))
            
    def forward(self, X):
        return self.act(tf.matmul(X, self.W) + self.b)
    
    def __call__(self, X):
        return self.forward(X)


SimpleLayer(3, 2)

In [34]:
sl = SimpleLayer(3, 2, bias=False)

sl.randomize()
sl.b, sl.W, sl.trainable_weights

(<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 0.]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
 array([[ 0.12896788,  0.15136927],
        [ 0.1670274 ,  0.05076873],
        [ 0.1074987 , -0.05629457]], dtype=float32)>,
 [<tf.Variable 'Variable:0' shape=(3, 2) dtype=float32, numpy=
  array([[ 0.12896788,  0.15136927],
         [ 0.1670274 ,  0.05076873],
         [ 0.1074987 , -0.05629457]], dtype=float32)>])

### IV.5. Custom Net

In [44]:
class SimpleMLP(object):
    def __init__(self):
        super().__init__()
        self.layers = []
        self.locked = False
        self.trainable_weights = []
    
    def add_layer(self, layer: SimpleLayer):
        if self.locked:
            raise Exception('You cannot add more layers')
        out_nbr = None
        if len(self.layers):
            out_nbr = self.layers[-1].W.shape[1]
        in_nbr = layer.W.shape[0]
        if out_nbr is not None and out_nbr != in_nbr:
            raise Exception(f'The last layer outputs ({out_nbr}) must be the same as this layer input {in_nbr}')
        self.layers.append(layer)
        self.trainable_weights.extend(layer.trainable_weights)
        return self
        
    def compile(self, nb_in=1, nb_out=1, bias=True, multiclass=False, lr=1.):
        if len(self.layers):
            nb_in = self.layers[-1].W.shape[1]
        out_layer = SimpleLayer(nb_in, nb_out, bias=bias, act='sigmoid')
        self.loss = SimpleBCE()
        if multiclass and nb_out > 1:
            out_layer.act = simple_softmax
            self.loss = SimpleCE()
        self.layers.append(out_layer)
        self.trainable_weights.extend(self.layers[-1].trainable_weights)
        self.optimizer = SimpleGD(learning_rate=lr)
        self.locked = True
        

    def forward(self, X):
        Z = X 
        for layer in self.layers:
            Z = layer(Z)
        return Z
    
    def backward(self, X, Y):
        with tf.GradientTape() as tape:
            Y_pred = self.forward(X)
            loss   = self.loss(Y_pred, Y)
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return loss.numpy()
    
    def fit(self, X, Y, epochs=20):
        for epoch in range(epochs):
            loss = self.backward(X, Y)
            print('epoch', epoch, ' loss =', loss)
    
    def randomize(self):
        for layer in self.layers:
            layer.randomize()
            
    def __call__(self, X):
        return self.forward(X)

In [45]:
# Result:
# tf.Tensor(
# [[0.8400944]
#  [0.8428117]], shape=(2, 1), dtype=float32)
# learning_rate 1.0
# 1.0020916
# <tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
# array([[0.51494634],
#        [0.5659208 ]], dtype=float32)>

nn3t = SimpleMLP()
nn3t.add_layer(SimpleLayer(2, 2, act='sigmoid'))\
    .add_layer(SimpleLayer(2, 2, act='sigmoid'))\
    .compile()


nn3t.layers[0].W.assign_add(tf.constant([[0.5, 0.3], [0.2, 0.4]]))
nn3t.layers[0].b.assign_add(tf.constant([[-0.3, 0.5]]))
nn3t.layers[1].W.assign_add(tf.constant([[0.3, -0.1], [0.5, -0.3]]))
nn3t.layers[1].b.assign_add(tf.constant([[-0.3, -0.2]]))
nn3t.layers[2].W.assign_add(tf.constant([[0.7], [0.7]]))
nn3t.layers[2].b.assign_add(tf.constant([[1.]]))

XX = tf.constant([[2, -1], [3, 5]], dtype=tf.float32)
YY = tf.constant([[0], [1]], dtype=tf.float32)

print(nn3t.forward(XX))

loss = nn3t.backward(XX, YY)

print(loss)

nn3t.layers[2].W

tf.Tensor(
[[0.8400944]
 [0.8428117]], shape=(2, 1), dtype=float32)
1.0020916


<tf.Variable 'Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[0.51494634],
       [0.5659208 ]], dtype=float32)>

### IV.6. Model training

In [52]:
nn3 = SimpleMLP()
nn3.add_layer(SimpleLayer(X_train.shape[1], 10, act='relu'))\
   .add_layer(SimpleLayer(10, 10, act='relu'))\
   .compile(nb_out=Y_train.shape[1], lr=0.01, multiclass=True)

nn3.randomize()

list(nn3.trainable_weights)

[<tf.Variable 'Variable:0' shape=(36, 10) dtype=float32, numpy=
 array([[-6.66655302e-02,  1.96015224e-01,  5.63323200e-02,
         -1.79208219e-01, -4.44375537e-02,  1.66296229e-01,
         -1.22174762e-01, -3.98689769e-02,  1.00832187e-01,
          3.11151445e-02],
        [-4.08484638e-02, -2.77882546e-01, -3.20380728e-04,
          2.31242720e-02, -1.48834646e-01, -6.28730133e-02,
          7.44477734e-02,  1.14146188e-01,  5.29693626e-02,
         -2.03437164e-01],
        [ 2.31579441e-04,  5.83794191e-02, -5.86487763e-02,
          7.38753239e-03, -2.07089689e-02, -1.12129115e-01,
         -5.65089360e-02,  7.56448805e-02,  8.03753212e-02,
         -8.61964822e-02],
        [-1.85606569e-01,  9.76005644e-02, -1.31842539e-01,
         -3.62384580e-02,  3.93696688e-02, -5.68981282e-02,
         -6.57454953e-02,  8.87177419e-03, -1.68923706e-01,
          1.85986459e-01],
        [-4.52180989e-02,  1.30691007e-01,  2.37443168e-02,
          6.86621740e-02,  5.31689525e-02,  1.52

In [53]:
nn3.fit(X_train, Y_train, epochs=10000)

epoch 0  loss = 0.29845637
epoch 1  loss = 0.2984484
epoch 2  loss = 0.29844043
epoch 3  loss = 0.29843247
epoch 4  loss = 0.2984245
epoch 5  loss = 0.29841658
epoch 6  loss = 0.29840863
epoch 7  loss = 0.29840064
epoch 8  loss = 0.29839268
epoch 9  loss = 0.29838476
epoch 10  loss = 0.2983768
epoch 11  loss = 0.29836884
epoch 12  loss = 0.29836088
epoch 13  loss = 0.29835293
epoch 14  loss = 0.29834497
epoch 15  loss = 0.29833704
epoch 16  loss = 0.2983291
epoch 17  loss = 0.29832113
epoch 18  loss = 0.29831317
epoch 19  loss = 0.2983052
epoch 20  loss = 0.2982973
epoch 21  loss = 0.2982893
epoch 22  loss = 0.29828137
epoch 23  loss = 0.2982734
epoch 24  loss = 0.2982655
epoch 25  loss = 0.2982575
epoch 26  loss = 0.29824954
epoch 27  loss = 0.2982416
epoch 28  loss = 0.29823363
epoch 29  loss = 0.29822567
epoch 30  loss = 0.29821774
epoch 31  loss = 0.2982098
epoch 32  loss = 0.29820186
epoch 33  loss = 0.2981939
epoch 34  loss = 0.29818597
epoch 35  loss = 0.298178
epoch 36  loss = 

### IV.7. Model testing

In [54]:
print(classification_report(Y_test, lbin.inverse_transform(nn3(X_test).numpy()), zero_division=0))

              precision    recall  f1-score   support

           1       0.23      1.00      0.37       461
           2       0.00      0.00      0.00       224
           3       0.00      0.00      0.00       397
           4       0.00      0.00      0.00       211
           5       0.00      0.00      0.00       237
           7       0.00      0.00      0.00       470

    accuracy                           0.23      2000
   macro avg       0.04      0.17      0.06      2000
weighted avg       0.05      0.23      0.09      2000

